In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import pickle

In [5]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Nyc-Taxi-Experiment")

<Experiment: artifact_location='/workspaces/Mlops-Zoomcamp/mlruns/1', creation_time=1717104848676, experiment_id='1', last_update_time=1717104848676, lifecycle_stage='active', name='Nyc-Taxi-Experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df["duration"]=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[((df.duration >= 1) & (df.duration <=60))]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df
    

In [7]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')


In [8]:
categorical = ["PU_DO"]
numerical = ['trip_distance']

In [9]:
df_train['PU_DO']=df_train["PULocationID"]+'_'+df_train["DOLocationID"]
df_val['PU_DO']=df_val["PULocationID"]+'_'+ df_val["DOLocationID"]

In [10]:
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical +numerical].to_dict(orient="records")

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'

y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False)

7.479562160810692

In [13]:
with mlflow.start_run():
    mlflow.set_tag("developer", "cristian")
    alpha = 0.1
    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "data/green_tripdata_2021-02.parquet")
    ls = Lasso(alpha = alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared = False)
    mlflow.log_metric("rmse", rmse)

In [14]:
with open("models/linreg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [17]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round=1000,
            evals = [(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared = False)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status":STATUS_OK}

In [18]:
"""search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),
    "learning_rate": hp.loguniform("learning_rate", -3,0),
    "reg_alpha": hp.loguniform("reg_alpha", -5,-1),
    "min_child_weight": hp.loguniform("min_child_weight", -1, 3),
    "objective": "reg:linear",
    "seed": 42
}

best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    max_evals= 50,
    trials = Trials()
)"""

'search_space = {\n    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),\n    "learning_rate": hp.loguniform("learning_rate", -3,0),\n    "reg_alpha": hp.loguniform("reg_alpha", -5,-1),\n    "min_child_weight": hp.loguniform("min_child_weight", -1, 3),\n    "objective": "reg:linear",\n    "seed": 42\n}\n\nbest_result = fmin(\n    fn = objective,\n    space = search_space,\n    algo = tpe.suggest,\n    max_evals= 50,\n    trials = Trials()\n)'

In [19]:
params = {
    "learning_rate": 0.27239573739603645,
    "max_depth": 17,
    "min_child_weight":1.172699044761734,
    "objective": "reg:linear",
    "reg_alpha":0.0616867958133725,
    "reg_lambda": 0.004264404814393109,
    "seed": 42
    
}

mlflow.xgboost.autolog()
booster = xgb.train(
    params = params,
    dtrain = train,
    num_boost_round=1000,
    evals = [(valid, "validatiaon")],
    early_stopping_rounds = 50
)

2024/05/31 07:20:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '46ce59938016415f9c9b81856037ecf7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


: 

: 

: 